<a href="https://colab.research.google.com/github/Sam-Batisti/mba-famesp-DataAnalytics/blob/main/Turnover_Data_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalar requerimentos Power BI

In [1]:
!pip install powerbiclient

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.9/684.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 71.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=86a9e20efc78eed462bd0073cb52f9517c2ef13c4694450417d286589d124110
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
from powerbiclient import Report, models
from powerbiclient.authentication import DeviceCodeLoginAuthentication


Importar Bibliotecas

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Requerimentos Drive

In [4]:
from google.colab import drive
from google.colab import output
drive.mount('/content/drive')

Mounted at /content/drive


Importar Dados


In [5]:
raw_data = pd.read_excel('/content/drive/MyDrive/MBA_Famesp/Data_Analytics/LEROY_CONECT.xlsx')
data_rh = pd.read_excel('/content/drive/MyDrive/MBA_Famesp/Data_Analytics/LEROY.xlsx')

Funções

In [6]:
def generate_metadata(df):
    metadata = pd.DataFrame({
        'nome_variavel': df.columns,
        'tipo': df.dtypes,
        'qt_nulos': df.isnull().sum(),
        'percent_nulos': round((df.isnull().sum()/len(df))*100,2),
        'cardinalidade': df.nunique(),
        'coluna_com_valores_unicos': df.nunique() == len(df),
    })
    metadata=metadata.sort_values(by='tipo')
    metadata=metadata.reset_index(drop=True)

    return metadata

def remover_colunas_se_existirem(df, colunas_a_remover, mensagem =True):
    for col in colunas_a_remover:
        if col in df.columns:
            df = df.drop(columns=[col])
            if mensagem:
                print(f"A coluna '{col}' foi removida.")
        else:
            if mensagem:
                print(f"A coluna '{col}' não foi encontrada no DataFrame.")
    return df

Exploratory Data Analysis

In [7]:
#Ler raw_data
raw_data.columns

Index(['DATE', 'YEAR', 'MONTH_NAME', 'Nome', 'RE', 'STATUS_MAPA',
       'STATUS_GERENCIAL', 'Supervisor', 'Celula', 'Calls_In', 'AHT',
       'ACDTIME', 'ACWTIME', 'HOLDTIME', 'TEMPO_FALADO', 'Calls_Out', 'OAHT',
       'AvailableTime', 'Aux0', 'Aux1', 'Aux2', 'Aux3', 'Aux4', 'Aux5', 'Aux6',
       'Aux7', 'Aux8', 'Aux9', 'HorasEscaladas', 'HorasABS', 'Adherence',
       'HorasADR', 'Folga', 'Escalados', 'DIA_LOGADO', 'Exceeded_Aux1',
       'Exceeded_Aux2', 'Escala'],
      dtype='object')

In [8]:
#Verificar qualidade dos dados da base raw_data
metadata_df = generate_metadata(raw_data)
metadata_df

,nome_variavel,tipo,qt_nulos,percent_nulos,cardinalidade,coluna_com_valores_unicos
0,DATE,datetime64[ns],0,0.00,50,False
1,YEAR,int64,0,0.00,1,False
2,RE,int64,0,0.00,110,False
3,Aux4,float64,2258,50.14,1,False
4,Aux5,float64,2258,50.14,885,False
5,Aux6,float64,2258,50.14,831,False
6,Aux7,float64,2258,50.14,850,False
7,Aux8,float64,2258,50.14,1,False
8,Aux9,float64,2258,50.14,1,False
9,HorasEscaladas,float64,676,15.01,5,False


In [9]:
#Ler data_rh
data_rh

,Global ID,Matricula,Descr.CCusto,Nome complet,Bairro,Estado,Municipio,Cep,Data Admis.,Dt. Demissao,Tipo Desligamento,Hrs. Mensais,Cod. Funcao,Desc.Funcao
0,1699351.0,45696,OPER LEROY MERLIN,ADRIANA GRANATO,VILA CRETY,SP,CARAPICUIBA,06386-000,2023-07-03,/ /,NaN,180,353,ANALISTA DE ATEND
1,1699111.0,45739,OPER LEROY MERLIN,ALESSANDRA NERY SANTOS,VILA VIRGINIA,SP,ITAQUAQUECETUBA,08575-030,2023-07-03,/ /,NaN,180,353,ANALISTA DE ATEND
2,1699112.0,45691,OPER LEROY MERLIN,ALEXANDRA MENDES VIEIRA GARCIA,HELENA MARIA,SP,OSASCO,06253-270,2023-07-03,/ /,NaN,180,353,ANALISTA DE ATEND
3,NaN,46138,OPER LEROY MERLIN,ALINE DE CARVALHO FERREIRA,VILA SUZANA,SP,SUZANO,08600-000,2023-09-01,/ /,NaN,180,353,ANALISTA DE ATEND
4,1699352.0,45719,OPER LEROY MERLIN,AMANDA DA SILVA GOMES,ITAIM PAULISTA,SP,SAO PAULO,08142-003,2023-07-03,2023-07-07 00:00:00,Voluntário,180,353,ANALISTA DE ATEND
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,1699346.0,45715,OPER LEROY MERLIN,THIAGO DA SILVA SANT ANA,JARDIM MARISTEL,SP,SAO PAULO,02806-000,2023-07-03,2023-08-09 00:00:00,Voluntário,180,353,ANALISTA DE ATEND
113,1707292.0,46040,OPER LEROY MERLIN,VANESSA MENDONCA DOS SANTOS,VILA SAO RAFAEL,SP,GUARULHOS,07044-050,2023-08-14,/ /,NaN,180,353,ANALISTA DE ATEND
114,1699348.0,45698,OPER LEROY MERLIN,VICTOR VIEIRA DA SILVA,CONJUNTO HABITA,SP,SAO PAULO,08485-520,2023-07-03,2023-09-01 00:00:00,Voluntário,180,353,ANALISTA DE ATEND
115,1699350.0,45738,OPER LEROY MERLIN,VITORIA KEZIA FERNANDES MONTEIRO,PARI,SP,SAO PAULO,03033-000,2023-07-03,/ /,NaN,180,353,ANALISTA DE ATEND


In [10]:
#Verificar qualidade dos dados da base raw_data
metadata_df = generate_metadata(data_rh)
metadata_df

,nome_variavel,tipo,qt_nulos,percent_nulos,cardinalidade,coluna_com_valores_unicos
0,Matricula,int64,0,0.00,117,True
1,Global ID,float64,25,21.37,92,False
2,Data Admis.,datetime64[ns],0,0.00,19,False
3,Hrs. Mensais,int64,0,0.00,2,False
4,Cod. Funcao,int64,0,0.00,7,False
5,Descr.CCusto,object,0,0.00,1,False
6,Nome complet,object,0,0.00,116,False
7,Bairro,object,0,0.00,113,False
8,Estado,object,0,0.00,1,False
9,Municipio,object,0,0.00,19,False


Tratamento das bases e Join

In [11]:
# Converte a coluna 'Dt._Demissao' para o tipo datetime e verifica se há valores nulos
data_rh['Foi_Desligado'] = ~pd.to_datetime(data_rh['Dt. Demissao'], errors='coerce').isnull()

# Converte valores True/False para 1/0
data_rh['Foi_Desligado'] = data_rh['Foi_Desligado'].astype(int)

In [12]:
# Criar os campos calculados para o PBI

# Calculando as novas colunas de desempenho
raw_data['C_TEMPO_LOGADO_MEDIA'] = np.where(raw_data['DIA_LOGADO'] != 0, (raw_data['HorasADR'] / raw_data['DIA_LOGADO']) / 86400, 0)
raw_data['C_ABS'] = np.where(raw_data['HorasEscaladas'] != 0, raw_data['HorasABS'] / raw_data['HorasEscaladas'], 0)
raw_data['C_ADR'] = np.where(raw_data['HorasEscaladas'] != 0, raw_data['HorasADR'] / raw_data['HorasEscaladas'], 0)
raw_data['C_HORAS_EM_PAUSA'] = np.where(raw_data['DIA_LOGADO'] != 0, ((raw_data['Aux0'] + raw_data['Aux1'] + raw_data['Aux2'] + raw_data['Aux3'] + raw_data['Aux4'] + raw_data['Aux5'] + raw_data['Aux6'] + raw_data['Aux7'] + raw_data['Aux8'] + raw_data['Aux9']) / 86400) / raw_data['DIA_LOGADO'], 0)
raw_data['C_PAUSAS_NR17'] = np.where(raw_data['DIA_LOGADO'] != 0, ((raw_data['Aux6'] + raw_data['Aux7']) / 86400) / raw_data['DIA_LOGADO'], 0)
raw_data['C_PAUSA_PARTICULAR'] = np.where(raw_data['DIA_LOGADO'] != 0, (raw_data['Aux1'] / 86400) / raw_data['DIA_LOGADO'], 0)
raw_data['C_OUTRAS_PAUSAS'] = np.where(raw_data['DIA_LOGADO'] != 0, ((raw_data['Aux0'] + raw_data['Aux1'] + raw_data['Aux2'] + raw_data['Aux3'] + raw_data['Aux4'] + raw_data['Aux5'] + raw_data['Aux8'] + raw_data['Aux9']) / 86400) / raw_data['DIA_LOGADO'], 0)
raw_data['C_TEMPO_IMPRODUTIVO_MEDIA'] = np.where(raw_data['DIA_LOGADO'] != 0, ((raw_data['Aux0'] + raw_data['Aux3'] + raw_data['Aux4'] + raw_data['Aux5'] + raw_data['Aux6'] + raw_data['Aux7'] + raw_data['Aux8'] + raw_data['Aux9'] + raw_data['Exceeded_Aux1'] + raw_data['Exceeded_Aux2']) / 86400) / raw_data['DIA_LOGADO'], 0)
raw_data['C_AHT'] = np.where((raw_data['Calls_In'] != 0) & (raw_data['TEMPO_FALADO'] != 0), (raw_data['TEMPO_FALADO'] / raw_data['Calls_In']) / 86400, 0)
raw_data['C_HORAS_PRODUTIVAS'] = np.where(raw_data['DIA_LOGADO'] != 0, (((raw_data['HorasADR'] / raw_data['DIA_LOGADO']) / 86400) - raw_data['C_TEMPO_IMPRODUTIVO_MEDIA']), 0)
raw_data['C_TEMPO_DESLOGADO'] = np.where(raw_data['DIA_LOGADO'] != 0, ((raw_data['HorasABS'] / raw_data['DIA_LOGADO']) / 86400), np.nan)




In [13]:
# Renomear a coluna "RE" da base_conect para matricula.
raw_data.rename(columns={'RE': 'Matricula'}, inplace=True)


In [14]:
# Lista de colunas de desempenho que terão a média e a soma
colunas_desempenho = ['C_TEMPO_LOGADO_MEDIA', 'C_ABS', 'C_ADR', 'C_HORAS_EM_PAUSA',
                      'C_PAUSAS_NR17', 'C_PAUSA_PARTICULAR', 'C_OUTRAS_PAUSAS',
                      'C_TEMPO_IMPRODUTIVO_MEDIA', 'C_AHT', 'C_HORAS_PRODUTIVAS',
                      'C_TEMPO_DESLOGADO']

# Agregar dados por matrícula e calcular médias e somas separadamente
dados_agrupados = raw_data.groupby('Matricula')[colunas_desempenho].agg(['mean', 'sum']).reset_index()

# Renomear colunas para incluir sufixo "_media" ou "_soma"
dados_agrupados.columns = ['Matricula'] + [f'{col}_{estatistica}' for col, estatistica in dados_agrupados.columns[1:]]


In [15]:
dados_agrupados

,Matricula,C_TEMPO_LOGADO_MEDIA_mean,C_TEMPO_LOGADO_MEDIA_sum,C_ABS_mean,C_ABS_sum,C_ADR_mean,C_ADR_sum,C_HORAS_EM_PAUSA_mean,C_HORAS_EM_PAUSA_sum,C_PAUSAS_NR17_mean,...,C_OUTRAS_PAUSAS_mean,C_OUTRAS_PAUSAS_sum,C_TEMPO_IMPRODUTIVO_MEDIA_mean,C_TEMPO_IMPRODUTIVO_MEDIA_sum,C_AHT_mean,C_AHT_sum,C_HORAS_PRODUTIVAS_mean,C_HORAS_PRODUTIVAS_sum,C_TEMPO_DESLOGADO_mean,C_TEMPO_DESLOGADO_sum
0,38543,0.160360,8.018009,0.026785,1.339262,0.469347,23.467344,0.234281,11.714074,0.015318,...,0.218964,10.948183,0.234281,11.714074,0.092955,3.718191,-0.073921,-3.696065,0.013458,0.457581
1,38559,0.121934,5.974757,0.003855,0.188889,0.356879,17.487093,0.283154,13.874537,0.018550,...,0.264604,12.965602,0.283154,13.874537,0.071473,2.501543,-0.161220,-7.899780,0.001956,0.064537
2,40887,0.133082,6.521042,0.016933,0.829709,0.389510,19.085976,0.244732,11.991875,0.017787,...,0.226945,11.120289,0.244732,11.991875,0.131191,4.985277,-0.111650,-5.470833,0.008590,0.283484
3,41256,0.072114,3.605694,0.000000,0.000000,0.211065,10.553252,0.105618,5.280880,0.000000,...,0.105618,5.280880,0.105618,5.280880,0.000000,0.000000,-0.033504,-1.675185,0.000000,0.000000
4,41648,0.023839,1.168090,0.038810,1.901711,0.090336,4.426447,0.065027,3.186319,0.000000,...,0.065027,3.186319,0.065027,3.186319,0.000000,0.000000,-0.041188,-2.018229,0.018304,0.237951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,46155,0.101620,0.609722,0.362135,2.172807,0.385088,2.310526,0.035517,0.213102,0.012969,...,0.022548,0.135289,0.035014,0.210081,0.005880,0.029398,0.066607,0.399641,0.015201,0.045602
106,46156,0.142971,0.857824,0.171645,1.029868,0.541784,3.250702,0.050365,0.302187,0.020887,...,0.029477,0.176863,0.049786,0.298715,0.004196,0.025174,0.093185,0.559109,0.001970,0.007882
107,46157,0.197801,1.384606,0.038452,0.269167,0.749561,5.246930,0.040359,0.282512,0.025112,...,0.015246,0.106725,0.037935,0.265544,0.011316,0.090528,0.159866,1.119062,0.011838,0.071030
108,46158,0.000000,0.000000,0.857143,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,NaN,0.000000


In [16]:
df_join = pd.merge(data_rh, dados_agrupados, left_on='Matricula', right_on='Matricula', how='inner')

In [17]:
## Fazer um join apenas das colunas necessárias da tabela raw_data, sem repetições

# Criar um dicionário com as correspondências entre 'Matricula' e 'Celula'
matricula_celula_dict = dict(zip(raw_data['Matricula'], raw_data['Celula']))

# Usar a função map para adicionar a coluna 'Celula'com base na 'Matricula'
df_join['Celula'] = df_join['Matricula'].map(matricula_celula_dict)

In [18]:
# Renomear as colunas tirando espaço
novo_nome_colunas = {}
for coluna in df_join.columns:
    novo_nome = coluna.replace(' ', '_')
    novo_nome_colunas[coluna] = novo_nome

# Renomeie as colunas no DataFrame
df_completo = df_join.rename(columns=novo_nome_colunas)

df_completo.columns

Index(['Global_ID', 'Matricula', 'Descr.CCusto', 'Nome_complet', 'Bairro',
       'Estado', 'Municipio', 'Cep', 'Data_Admis.', 'Dt._Demissao',
       'Tipo_Desligamento', 'Hrs._Mensais', 'Cod._Funcao', 'Desc.Funcao',
       'Foi_Desligado', 'C_TEMPO_LOGADO_MEDIA_mean',
       'C_TEMPO_LOGADO_MEDIA_sum', 'C_ABS_mean', 'C_ABS_sum', 'C_ADR_mean',
       'C_ADR_sum', 'C_HORAS_EM_PAUSA_mean', 'C_HORAS_EM_PAUSA_sum',
       'C_PAUSAS_NR17_mean', 'C_PAUSAS_NR17_sum', 'C_PAUSA_PARTICULAR_mean',
       'C_PAUSA_PARTICULAR_sum', 'C_OUTRAS_PAUSAS_mean', 'C_OUTRAS_PAUSAS_sum',
       'C_TEMPO_IMPRODUTIVO_MEDIA_mean', 'C_TEMPO_IMPRODUTIVO_MEDIA_sum',
       'C_AHT_mean', 'C_AHT_sum', 'C_HORAS_PRODUTIVAS_mean',
       'C_HORAS_PRODUTIVAS_sum', 'C_TEMPO_DESLOGADO_mean',
       'C_TEMPO_DESLOGADO_sum', 'Celula'],
      dtype='object')

In [19]:
# Lista de colunas a serem removidas
colunas_a_remover = ['Global_ID', 'Nome_complet', 'Descr.CCusto', 'Cod._Funcao']
# Chamar função
df_completo = remover_colunas_se_existirem(df_completo, colunas_a_remover)

A coluna 'Global_ID' foi removida.
A coluna 'Nome_complet' foi removida.
A coluna 'Descr.CCusto' foi removida.
A coluna 'Cod._Funcao' foi removida.


In [20]:
# Verificar se há campos com valores únicos
metadata_df = generate_metadata(df_completo)
metadata_df

,nome_variavel,tipo,qt_nulos,percent_nulos,cardinalidade,coluna_com_valores_unicos
0,Matricula,int64,0,0.00,109,True
1,Foi_Desligado,int64,0,0.00,2,False
2,Hrs._Mensais,int64,0,0.00,2,False
3,C_HORAS_PRODUTIVAS_mean,float64,0,0.00,98,False
4,C_AHT_sum,float64,0,0.00,92,False
5,C_AHT_mean,float64,10,9.17,92,False
6,C_TEMPO_IMPRODUTIVO_MEDIA_sum,float64,0,0.00,98,False
7,C_TEMPO_IMPRODUTIVO_MEDIA_mean,float64,0,0.00,97,False
8,C_PAUSAS_NR17_mean,float64,0,0.00,89,False
9,C_OUTRAS_PAUSAS_sum,float64,0,0.00,98,False


In [21]:
# Confirmar que as colunas são do tipo datetime
df_completo['Data_Admis.'] = pd.to_datetime(df_completo['Data_Admis.'], errors='coerce')
df_completo['Dt._Demissao'] = pd.to_datetime(df_completo['Dt._Demissao'], errors='coerce')

# Converta colunas de datas em um formato de exibição específico
df_completo['Data_Admis.'] = df_completo['Data_Admis.'].dt.strftime('%Y-%m-%d')
df_completo['Dt._Demissao'] = df_completo['Dt._Demissao'].dt.strftime('%Y-%m-%d')

In [22]:
df_completo.describe()

,Matricula,Hrs._Mensais,Foi_Desligado,C_TEMPO_LOGADO_MEDIA_mean,C_TEMPO_LOGADO_MEDIA_sum,C_ABS_mean,C_ABS_sum,C_ADR_mean,C_ADR_sum,C_HORAS_EM_PAUSA_mean,...,C_OUTRAS_PAUSAS_mean,C_OUTRAS_PAUSAS_sum,C_TEMPO_IMPRODUTIVO_MEDIA_mean,C_TEMPO_IMPRODUTIVO_MEDIA_sum,C_AHT_mean,C_AHT_sum,C_HORAS_PRODUTIVAS_mean,C_HORAS_PRODUTIVAS_sum,C_TEMPO_DESLOGADO_mean,C_TEMPO_DESLOGADO_sum
count,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,...,109.000000,109.000000,109.000000,109.000000,99.000000,109.000000,109.000000,109.000000,97.000000,109.000000
mean,45530.146789,178.899083,0.201835,0.122681,4.352779,0.124147,4.077810,0.438174,15.162756,0.048136,...,0.029749,1.219637,0.047165,1.853055,0.011473,0.291830,0.075516,2.499724,0.010369,0.190136
std,1280.381872,5.666577,0.403224,0.069861,3.556106,0.188316,6.618181,0.252304,12.187786,0.043892,...,0.042616,2.157689,0.043861,2.306755,0.020959,0.731977,0.070116,3.225510,0.010583,0.226869
min,38543.000000,150.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.161220,-7.899780,0.000000,0.000000
25%,45707.000000,180.000000,0.000000,0.052197,1.043021,0.022224,0.425556,0.197800,3.952500,0.030662,...,0.011421,0.193553,0.029675,0.330347,0.005561,0.032560,0.024987,0.355637,0.004074,0.023704
50%,45740.000000,180.000000,0.000000,0.153386,3.760833,0.049750,1.755789,0.541784,13.590650,0.042270,...,0.020104,0.571042,0.040733,1.359410,0.007218,0.152193,0.104070,2.038600,0.007550,0.106759
75%,46046.000000,180.000000,0.000000,0.179199,8.106447,0.115300,3.986798,0.635051,27.710908,0.059535,...,0.032315,1.410301,0.057779,2.753067,0.009603,0.277332,0.129223,5.465000,0.014283,0.257384
max,46159.000000,180.000000,1.000000,0.217831,10.891539,0.857143,34.000000,0.749561,36.140351,0.283154,...,0.264604,12.965602,0.283154,13.874537,0.135261,4.985277,0.161386,8.069317,0.075718,1.243229


Tabela final para o PBI

In [33]:
df_completo.to_excel('/content/drive/MyDrive/MBA_Famesp/Data_Analytics/df_completo.xlsx',index=False)


In [24]:
# Importar DeviceCodeLoginAuthentication class para autenticação einicialização do PowerBI
device_auth = DeviceCodeLoginAuthentication()

Performing device flow authentication. Please follow the instructions below.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code EF3G66L4F to authenticate.

Device flow authentication successfully completed.
You are now logged in .

The result should be passed only to trusted code in your notebook.


In [25]:
# Workspace
group_id="baaf63ab-bc66-4b9f-b40e-b7c51708c9f2"
# Relatório
report_id="d83cdd94-74f5-4cfe-b3ed-1c72cc7f0ce0"
report = Report(group_id=group_id, report_id=report_id, auth=device_auth)
report.set_size(1024, 1600)
output.enable_custom_widget_manager()

In [26]:
# Analise em PowerBi
report

Report(container_height=1024.0, container_width=1600.0)